### Missing Photo Detect

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 28 native word end index

In [4]:
def word_usage_result(word_list, df_target, target_column, word_usage_min, word_usage_max): # word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max)
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        #opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value])
            #result_list_select.append([target_value,opt_value])  
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}"])
    #df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    #df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [5]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [6]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [7]:
image_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/40000 Words/Images_Crop_Size"
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/Data/Deployment"
#file = "Talk Time 6 Video List.xlsx"
#sheet = "Sheet1"  # Sheet1

In [ ]:
#df_video_file = pd.read_excel(f"{path_folder}/{file}")
#df_video_file

In [10]:
ngram_video_file_list = glob.glob(f"{path_folder}/Talk Time 6 * Video List.xlsx")
ngram_video_file_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Twogram Video List.xlsx',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Threegram Video List.xlsx',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Fourgram Video List.xlsx',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Fivegram Video List.xlsx',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Sixgram Video List.xlsx',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 6/Data/Deployment/Talk Time 6 Sevengram Video List.xlsx']

In [19]:
df_video_file = pd.DataFrame()
ngram_num = 1
for file in ngram_video_file_list:
    df_var = pd.read_excel(f"{file}")
    df_var = df_var.iloc[:,[0]]
    old_name = df_var.columns[0]
    df_var = df_var.rename(columns={f"{old_name}":f"ngram{ngram_num}"})
    df_video_file = pd.concat([df_video_file,df_var], axis=1)
    ngram_num +=1 
df_video_file

,ngram1,ngram2,ngram3,ngram4,ngram5,ngram6
0,her gün,kısa bir süre,bir şey söyleyeyim mi,bir şey varsa o da,bana söylemek istediğin bir şey var,söylemek istediğin başka bir şey var mı
1,çok önemli,çok memnun oldum,benim için çok önemli,sana bir şey söyleyeyim mi,daha önce hiç böyle bir şey,bir iyi bir de kötü haberim var
2,burada bir,çok ama çok,öyle ya da böyle,bilmem gereken bir şey var,hakkında en ufak bir fikrim yok,NaN
3,tane daha,ne anlama geldiğini,elimden gelenin en iyisini,bize biraz izin verir misiniz,istediğiniz başka bir şey var mı,NaN
4,bir yere,bir süre sonra,gereken çok şey var,bu iyi bir şey değil,NaN,NaN
...,...,...,...,...,...,...
67,ihtiyacı var,NaN,NaN,NaN,NaN,NaN
68,kusura bakma,NaN,NaN,NaN,NaN,NaN
69,kusura bakmayın,NaN,NaN,NaN,NaN,NaN
70,mümkün değil,NaN,NaN,NaN,NaN,NaN


In [20]:
df_file_count = word_count_result(df_video_file,df_video_file.columns)
df_file_count

,word,word_count
0,bir,63
1,şey,33
2,ne,23
3,bu,16
4,var,12
...,...,...
176,haberim,1
177,görüyor,1
178,görmek,1
179,gördünüz,1


In [25]:
image_file_list = os.listdir(image_path_folder)
filename_without_ext_list = []
for file in image_file_list:
    # file and extention
    file_without_ext = os.path.splitext(file)[0]
    filename_without_ext_list.append(file_without_ext)
#filename_without_ext_list

In [23]:
set_image_file = set(filename_without_ext_list)
set_video_file = set(df_file_count["word"])

In [24]:
set_video_file.difference(set_image_file)

{'acil',
 'altında',
 'aynen',
 'az',
 'bakma',
 'bakmayın',
 'bilmem',
 'bitti',
 'buldum',
 'bırak',
 'durum',
 'duydun',
 'dürüst',
 'düşünüyorsun',
 'edin',
 'fena',
 'geldin',
 'gelenin',
 'gelmek',
 'gerekirse',
 'gördünüz',
 'görüyor',
 'haberim',
 'halde',
 'hızlı',
 'ihtiyacı',
 'istediğini',
 'iyisini',
 'kadın',
 'kere',
 'kontrol',
 'korkunç',
 'kulağa',
 'kusura',
 'mümkün',
 'neydi',
 'neyse',
 'olmamış',
 'olun',
 'soracağım',
 'sorunun',
 'söyleyeceğim',
 'söyleyeyim',
 'tabi',
 'varsa',
 'verir',
 'yapmak',
 'yaptığını',
 'yerde',
 'yere',
 'zorunda',
 'önemi',
 'özel'}